Welcome to  nemo swahili model   train  code run  the  following  cells in the order  indicated e.g  
#first step to  run
 above  will  be in each  cell


Below cell it will prompt you to connect to google  drive  click connect and select  the  account and allow

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

After   connect  the 

In [ ]:
!tar -xzvf "/content/drive/Mozilla/eval0.tar.gz" -C "/content/drive/MyDrive/Mozilla/clips/"     #[run this cell to extract tar.gz files]
!tar -xzvf "/content/drive/Mozilla/test0.tar.gz" -C "/content/drive/MyDrive/Mozilla/clips/"     #[run this cell to extract tar.gz files]
!tar -xzvf "/content/drive/Mozilla/train0.tar.gz" -C "/content/drive/MyDrive/Mozilla/clips/"     #[run this cell to extract tar.gz files]

In [ ]:
#second  step  run this  step  after

!pip install --upgrade pip
!pip install wget text-unidecode
!python -m pip install git+https://github.com/NVIDIA/NeMo.git
!pip install nemo_toolkit['all']
!apt-get update && apt-get install -y libsndfile1 ffmpeg sox   libsox-fmt-mp3
!pip install Cython
!sudo apt-get update
!sudo apt-get upgrade
!sudo apt-get install lame
!pip install sox lameenc jsonlines
!pip install ffmpeg-python wget tensorflow
!pip install text-unidecode pydub
!pip install matplotlib>=3.3.2
# Install NeMo
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
!pip install tensorflow[and-cuda]

Above cell  is of the utmost  importance it need to be ran second after cell 0 since in install all required libraries  nemo requires and other files  require eg ffmpeg 

Below 4 cells are optional cells to  run if  you  don't have error with graphics on an issues are to be ran  if there is an issues  with your graphics

In [ ]:
# third  step run this  as  the third  step
# Important  this cell on running  it will require  you to press  the  output  cell and hit enter to continue

# install nvidia & other dependencies
!add-apt-repository ppa:graphics-drivers/ppa
!apt update
!apt install nvidia-384 nvidia-384-dev
!apt-get install g++ freeglut3-dev build-essential libx11-dev libxmu-dev libxi-dev libglu1-mesa libglu1-mesa-dev

In [ ]:
#step  five

# installing cuda

!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/cuda-ubuntu2204.pin
!sudo mv cuda-ubuntu2204.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/12.3.0/local_installers/cuda-repo-ubuntu2204-12-3-local_12.3.0-545.23.06-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu2204-12-3-local_12.3.0-545.23.06-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu2204-12-3-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda-toolkit-12-3

In [ ]:
#step  six
# # driver installer
!sudo apt-get install -y cuda-drivers

In [ ]:
# step  seven
# # nvidia toolkit cuda
!apt-get install nvidia-cuda-toolkit

above 4 cells  respectively are to be ran if you find  error in graphics  error so if there is no error skip the cells

In [ ]:
# step eight  to  show  if you are connected to the gpu
import tensorflow as tf
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

In [ ]:
#fourth step  run this as 4th  step
# show our  gpu
!nvidia-smi

Above  2 cells is important to see if you have instantiated gpu  in colab or machine has gpu support

Below is code to convert .tsv to json file

# if you  didn't  have any issue and want  to continue  training  or transcribe the  files  skip  the following cells since  have prepared  the dataset skip till where  you find start  here  to train then transcribe

3 cells below is  to convert tsv and csv  files  respectively and directing them to  where their  files are located

In [ ]:
!python drive/MyDrive/Mozilla/tsv_to_json.py \
  --tsv=drive/MyDrive/Mozilla/train.tsv \
  --folder=/content/drive/MyDrive/Mozilla/clips/train \
  --sampling_count=-1

In [ ]:
!python drive/MyDrive/Mozilla/csv_to_json.py \
  --csv=drive/MyDrive/Mozilla/eval.csv \
  --folder=/content/drive/MyDrive/Mozilla/clips/eval \
  --sampling_count=-1

In [ ]:
!python drive/MyDrive/Mozilla/csv_to_json.py \
  --csv=drive/MyDrive/Mozilla/test.csv \
  --folder=/content/drive/MyDrive/Mozilla/clips/test \
  --sampling_count=-1

Below  3 cells  are to resample our files  from .mp3 to .wav with sample rate of 16000hz

In [ ]:
!mkdir drive/MyDrive/Mozilla/train_data/
!python drive/MyDrive/Mozilla/decode_resample.py \
  --manifest=drive/MyDrive/Mozilla/train.json \
  --destination_folder=drive/MyDrive/Mozilla/train_data

In [ ]:
!mkdir drive/MyDrive/Mozilla/eval_data/
!python drive/MyDrive/Mozilla/csv_resample.py \
  --manifest=drive/MyDrive/hackathon/eval.json \
  --destination_folder=/content/drive/MyDrive/Mozilla/eval_data

In [ ]:
!mkdir drive/MyDrive/Mozilla/test_data/
!python drive/MyDrive/Mozilla/csv_resample.py \
  --manifest=drive/MyDrive/Mozilla/test.json \
  --destination_folder=drive/MyDrive/Mozilla/test_data

This  cell below  in simple terms we clean our audio files of unnecessary  character  eg special characters

In [ ]:
# prepare_dataset_kiswahili.py
import json
import os
import re
from collections import defaultdict
from nemo.collections.asr.parts.utils.manifest_utils import read_manifest, write_manifest
from tqdm.auto import tqdm

def write_processed_manifest(data, original_path):
    original_manifest_name = os.path.basename(original_path)
    new_manifest_name = original_manifest_name.replace(".json", "_processed.json")

    manifest_dir = os.path.split(original_path)[0]
    filepath = os.path.join(manifest_dir, new_manifest_name)
    write_manifest(filepath, data)
    print(f"Finished writing manifest: {filepath}")
    return filepath


# calculate the character set
def get_charset(manifest_data):
    charset = defaultdict(int)
    for row in tqdm(manifest_data, desc="Computing character set"):
        text = row['text']
        for character in text:
            charset[character] += 1
    return charset


# Preprocessing steps

def remove_special_characters(data):
    chars_to_ignore_regex = "[\.\,\?\:\-!;()«»…\]\[/\*–‽+&_\\½√>€™$•¼}{~—=“\"”″‟„]"
    apostrophes_regex = "[’'‘`ʽ']"
    # print(data["text"])
    if data is not None and isinstance(data, dict) and "text" in data:
        text = data["text"]

        # Check if text is not empty
        if text:
            # Chain regular expressions for better performance
            cleaned_text = re.sub(chars_to_ignore_regex, " ", text)
            cleaned_text = re.sub(apostrophes_regex, "'", cleaned_text)
            cleaned_text = re.sub(r"'+", "'", cleaned_text)
            cleaned_text = re.sub(r"([b-df-hj-np-tv-z])' ([aeiou])", r"\1'\2", cleaned_text)
            cleaned_text = re.sub(r" '| '", " ", cleaned_text)
            cleaned_text = re.sub(r"' ", " ", cleaned_text)
            cleaned_text = re.sub(r" +", " ", cleaned_text)

            data["text"] = cleaned_text
        else:
            print(f"{data['text']} is empty")
    else:
        print("data is None or not a dictionary")

    return data



# a             ā, ã
# e             ē
# i             ī, i̇
# o             ō
# u             ū
# c             č
# //g             ğ
# //k             ǩ
# //s             š
#// t             ŧ
#// y             ȳ
def replace_diacritics(data):
    data["text"] = re.sub(r"[éèëēê]", "e", data["text"])
    data["text"] = re.sub(r"[ãâāá]", "a", data["text"])
    data["text"] = re.sub(r"[úūü]", "u", data["text"])
    data["text"] = re.sub(r"[ôōó]", "o", data["text"])
    data["text"] = re.sub(r"[ćç]", "c", data["text"])
    data["text"] = re.sub(r"[ïī]", "i", data["text"])
    data["text"] = re.sub(r"[ñ]", "n", data["text"])
    data["text"] = re.sub(r"[ŧ]", "t", data["text"])
    data["text"] = re.sub(r"[ȳ]", "y", data["text"])
    data["text"] = re.sub(r"[š]", "s", data["text"])
    data["text"] = re.sub(r"[ǩ]", "k", data["text"])
    data["text"] = re.sub(r"[ğ]", "g", data["text"])
    return data


def remove_oov_characters(data):
    oov_regex = "[^ 'aiuenrbomkygwthszdcjfvplxq]"
    data["text"] = re.sub(oov_regex, "", data["text"])  # delete oov characters
    data["text"] = data["text"].strip()
    return data


# Processing pipeline
def apply_preprocessors(manifest, preprocessors):
    for processor in preprocessors:
        for idx in tqdm(range(len(manifest)), desc=f"Applying {processor.__name__}"):
            manifest[idx] = processor(manifest[idx])

    print("Finished processing manifest !")
    return manifest


# List of pre-processing functions
PREPROCESSORS = [
    remove_special_characters,
    replace_diacritics,
    remove_oov_characters,
]

train_manifest = "./drive/MyDrive/Mozilla/train_decoded.json"
eval_manifest = "./drive/MyDrive/Mozilla/eval_decoded.json"
test_manifest = "./drive/MyDrive/Mozilla/test_decoded.json"

train_data = read_manifest(train_manifest)
eval_data = read_manifest(eval_manifest)
test_data = read_manifest(test_manifest)



# #Apply preprocessing
train_data_processed = apply_preprocessors(train_data, PREPROCESSORS)
eval_data_processed = apply_preprocessors(eval_data, PREPROCESSORS)
test_data_processed = apply_preprocessors(test_data, PREPROCESSORS)

## Write new manifests
train_manifest_cleaned = write_processed_manifest(train_data_processed, train_manifest)
eval_manifest_cleaned = write_processed_manifest(eval_data_processed, eval_manifest)
test_manifest_cleaned = write_processed_manifest(test_data_processed, test_manifest)


Create a tokenized data to improve  quality and speed.

In [ ]:
# #Tokenizers
!python  drive/MyDrive/Mozilla/scripts/process_asr_text_tokenizer.py \
  --manifest=drive/MyDrive/Mozilla/eval_decoded_processed.json,drive/MyDrive/Mozilla/train_decoded_processed.json \
  --vocab_size=1024 \
  --data_root=./drive/MyDrive/Mozilla/ \
  --tokenizer="spe" \
  --spe_type=bpe \
  --spe_character_coverage=1.0 \
  --spe_max_sentencepiece_length=4 \
  --log



To download respective  scripts may omit  they are available  in the  drive  link

In [ ]:
BRANCH = 'main'
!wget -P drive/MyDrive/Mozilla/scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/scripts/speech_recognition/convert_to_tarred_audio_dataset.py
!wget -P drive/MyDrive/Mozilla/scripts/ https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/asr_ctc/speech_to_text_ctc_bpe.py


This  cell will produce  one bucket of tarred  dataset  this will improve the speed of training

In [ ]:
# # Tarred datasets and bucketing

# create tarred dataset with 1 bucket
!python drive/MyDrive/Mozilla/scripts/convert_to_tarred_audio_dataset.py \
  --manifest_path=drive/MyDrive/Mozilla/train_decoded_processed.json \
  --target_dir=drive/MyDrive/Mozilla/train_tarred_1bk \
  --num_shards=1024 \
  --max_duration=11.0 \
  --min_duration=1.0 \
  --shuffle \
  --shuffle_seed=1 \
  --sort_in_shards \
  --workers=-1



<b>Start Here   to train and next cells to transcribe</b>


Below is the final  cell to run run to continue  training
run  this  cell  to train the model
After  it has  started to  execute  check  to  this  if the  will print
Restoring all states  from SwahiliModelTraining1--val_wer=inf-epoch=9-last.ckpt   something like  this.
It  will take approximate  of  2 and 30 minss

In [ ]:
# Training scripts and configs
# /content/drive/MyDrive/Mozilla/train_tarred_1bk
!python drive/MyDrive/Mozilla/scripts/speech_to_text_ctc_bpe.py \
--config-path=../conf/conformer/ \
--config-name=conformer_ctc_bpe \
exp_manager.name="SwahiliModelTraining1" \
exp_manager.resume_if_exists=true \
exp_manager.resume_ignore_no_checkpoint=true \
exp_manager.exp_dir=drive/MyDrive/Mozilla/ \
model.tokenizer.dir=drive/MyDrive/Mozilla/tokenizer_spe_bpe_v1024_max_4 \
model.train_ds.is_tarred=true \
model.train_ds.tarred_audio_filepaths=drive/MyDrive/Mozilla/train_tarred_1bk/audio__OP_0..1023_CL_.tar \
model.train_ds.manifest_filepath=drive/MyDrive/Mozilla/train_tarred_1bk/tarred_audio_manifest.json \
model.validation_ds.manifest_filepath=/content/drive/MyDrive/Mozilla/eval_decoded_processed.json \
model.test_ds.manifest_filepath=drive/MyDrive/Mozilla/test_decoded_processed_v2.json


The  following  cell is to transcribe the test  data and produce  thejson file with description

In [ ]:
# running interefence one
# /content/drive/MyDrive/Mozilla/SwahiliModelTraining/checkpoints/SwahiliModelTraining.nemo
!python drive/MyDrive/Mozilla/scripts/transcribe_speech.py \
  model_path=/content/drive/MyDrive/Mozilla/SwahiliModelTraining1/checkpoints/SwahiliModelTraining1.nemo \
  dataset_manifest=drive/MyDrive/Mozilla/test_decoded_processed_v2.json \
  output_filename=drive/MyDrive/Mozilla/results/test_with_predictions_4.json \
  batch_size=16 \
  cuda=0 \
  amp=True

Below  Code is to transcribe  the eval audio  for submission

In [ ]:


!python drive/MyDrive/Mozilla/scripts/transcribe_speech.py \
  model_path=/content/drive/MyDrive/Mozilla/SwahiliModelTraining1/checkpoints/SwahiliModelTraining1.nemo \
  dataset_manifest=drive/MyDrive/Mozilla/Prime_eval.json \
  output_filename=drive/MyDrive/Mozilla/results/eval_version1.json \
  batch_size=16 \
  cuda=0 \
  amp=True

The Following  Batch 1 Batch 2 and Batch 3  are chunk of eval data i chunked to decrease the load on the transcribe script  i then combine them with the code  in the end.

In [ ]:
# BATCH 1

!python drive/MyDrive/Mozilla/scripts/transcribe_speech.py \
  model_path=/content/drive/MyDrive/Mozilla/SwahiliModelTraining1/checkpoints/SwahiliModelTraining1.nemo \
  dataset_manifest=drive/MyDrive/Mozilla/eval_decoded_processed_1.json \
  output_filename=drive/MyDrive/Mozilla/results/eval_version3_1.json \
  batch_size=16 \
  cuda=0 \
  amp=True


In [ ]:
# BATCH 2
!python drive/MyDrive/Mozilla/scripts/transcribe_speech.py \
  model_path=/content/drive/MyDrive/Mozilla/SwahiliModelTraining1/checkpoints/SwahiliModelTraining1.nemo \
  dataset_manifest=drive/MyDrive/Mozilla/eval_decoded_processed_2.json \
  output_filename=drive/MyDrive/Mozilla/results/eval_version3_2.json \
  batch_size=16 \
  cuda=0 \
  amp=True

In [ ]:
# BATCH 3
# /content/drive/MyDrive/Mozillafggg
!python /content/drive/MyDrive/Mozilla/scripts/transcribe_speech.py \
  model_path=/content/drive/MyDrive/Mozilla/SwahiliModelTraining1/checkpoints/SwahiliModelTraining1.nemo \
  dataset_manifest=/content/drive/MyDrive/Mozilla/eval_decoded_processed_3.json \
  output_filename=/content/drive/MyDrive/Mozilla/results/eval_version3_3.json \
  batch_size=16 \
  cuda=0 \
  amp=True

The  Code below  is to combine different json files to  one and  remove any duplicates maybe  found/

In [ ]:
import os
import pandas as pd
import jsonlines
import csv

###### List of input JSONL files to combine

input_files = ["eval_version3_1.json","test_with_predictions_4.json",]

###### Output file where the combined JSONL will be stored

file=r'drive/MyDrive/Mozilla/results/final_data.json'

###### Function to combine multiple JSONL files

def combine_jsonl(input_files, output_file):
    with jsonlines.open(output_file, 'w') as writer:
        for input_file in input_files:
            file=f'drive/MyDrive/Mozilla/results/{input_file}'
            with jsonlines.open(file) as reader:
                for line in reader:
                    writer.write(line)

###### Combine the JSONL files

combine_jsonl(input_files, file)

##### Now to extract the paths
JsonPath =r'drive/MyDrive/Mozilla/results/final_data.json'

folder_path = r'drive/MyDrive/Mozilla/results/'

csvName = "eval_with_predictions.json"
data_list = {}  # List to store the JSON data

sentence = []  # This list will store the processed data
path = []  # This list will store the processed data

with jsonlines.open(JsonPath) as reader:
    for obj in reader:
        file_path = os.path.basename(obj['audio_filepath'])
        file_name = os.path.splitext[file_path](0) + ".mp3"
        path.append(file_name)  # the non-empty row to the output_data list
        sentence.append(0 if not obj['pred_text'].strip() else obj['pred_text'])# the non-empty row to the output_data list
SampeData = {
    "path":path,
    "sentence":sentence,

}
df = pd.DataFrame(SampeData)
folder_path = r'drive/MyDrive/Mozilla/results/'
df.to_csv(os.path.join(folder_path,'final_data.csv'),index=False)

### This to remove duplicates incase there is 
def remove_duplicates(csv_path, column_name):
    # Check if the file exists
    if not os.path.exists(csv_path):
        print(f"File not found: {csv_path}")
        return
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_path)
    # Remove duplicates based on the specified column
    df_no_duplicates = df.drop_duplicates(subset=[column_name])
    # Save the DataFrame with duplicates removed
    df_no_duplicates.to_csv(csv_path, index=False)
    print(f"Duplicates removed from {csv_path}")

# Example usage
csv_path = r'drive/MyDrive/Mozilla/results/final_data.csv'
column_to_check = 'path'
remove_duplicates(csv_path, column_to_check)

Note  I found my prepared dataset didn't  have the following  files
```common_voice_sw_28266617.mp3
common_voice_sw_37214113.mp3
common_voice_sw_37664539.mp3
common_voice_sw_31290838.mp3
common_voice_sw_35087387.mp3```